In [3]:
import pandas as pd


In [7]:
columns =['input_dt',  'feed_plan_cd','feed_plan_nm', 'farm_no', 'farm_pig_no', 'feed_plan_pct', 'gyobae_dt', 'bunman_dt', 'wk_gubun', 'wk_dt','one_intake_kg', 'one_tm','one_kg', 'one_now_kg', 'two_tm', 'two_kg', 'two_now_kg', 'three_tm','three_kg', 'three_now_kg', 'four_tm', 'four_kg', 'four_now_kg','five_tm', 'five_kg', 'five_now_kg', 'warn_kg', 'min_kg', 'log_dt']  ##컬럼이름 설정
data_columns = columns + ['next_bunman_sancha','next_jae','next_chongsan','next_silsan','next_eudusu']  #컬럼추가
data = pd.DataFrame(columns=data_columns) ###추가된 컬럼



########################################################################################
ba_pi = pd.read_csv('/mnt/d/project/comars/pigplan_1387.csv')
ba_co = pd.read_csv('/mnt/d/project/comars/uit_feeding_hy_1387_바른양돈.csv')
print(data)
###################################
ba_co = ba_co[columns]
pig_list = ba_pi['FARM_PIG_NO'].unique()
co_list = ba_co['farm_pig_no'].unique()

Empty DataFrame
Columns: [input_dt, feed_plan_cd, feed_plan_nm, farm_no, farm_pig_no, feed_plan_pct, gyobae_dt, bunman_dt, wk_gubun, wk_dt, one_intake_kg, one_tm, one_kg, one_now_kg, two_tm, two_kg, two_now_kg, three_tm, three_kg, three_now_kg, four_tm, four_kg, four_now_kg, five_tm, five_kg, five_now_kg, warn_kg, min_kg, log_dt, next_bunman_sancha, next_jae, next_chongsan, next_silsan, next_eudusu]
Index: []

[0 rows x 34 columns]


In [6]:
a = ['s','b', 'c']
print(a)
c = a+ ['d' , 'e']

print(c)

['s', 'b', 'c']
['s', 'b', 'c', 'd', 'e']


In [4]:

columns =['input_dt',  'feed_plan_cd','feed_plan_nm', 'farm_no', 'farm_pig_no', 'feed_plan_pct', 'gyobae_dt', 'bunman_dt', 'wk_gubun', 'wk_dt','one_intake_kg', 'one_tm','one_kg', 'one_now_kg', 'two_tm', 'two_kg', 'two_now_kg', 'three_tm','three_kg', 'three_now_kg', 'four_tm', 'four_kg', 'four_now_kg','five_tm', 'five_kg', 'five_now_kg', 'warn_kg', 'min_kg', 'log_dt']
data_columns = columns + ['next_bunman_sancha','next_jae','next_chongsan','next_silsan','next_eudusu']
data = pd.DataFrame(columns=data_columns)
ba_pi = pd.read_csv('/mnt/d/project/comars/pigplan_1387.csv')
ba_co = pd.read_csv('/mnt/d/project/comars/uit_feeding_hy_1387_바른양돈.csv')
ba_co = ba_co[columns]
pig_list = ba_pi['FARM_PIG_NO'].unique()
co_list = ba_co['farm_pig_no'].unique()
for pig_id in tqdm(pig_list):
    if pig_id in co_list:
        tmp_co = ba_co[ba_co['farm_pig_no'] == pig_id]
        tmp_pi = ba_pi[ba_pi['FARM_PIG_NO'] == pig_id]
        gyobaes = list(tmp_pi['GYOBAE_DT'].unique())
        gyobaes.sort()
        gyobaes_list = []
        for i in gyobaes:
            tmp_gyo = str(i)
            gyo_1 = tmp_gyo[:4]
            gyo_2 = tmp_gyo[4:6]
            gyo_3 = tmp_gyo[6:]
            gyobaes_list.append(gyo_1 + '-' + gyo_2 + '-' +gyo_3)
        for gyobae_num in range(len(gyobaes_list)):
            tmp = tmp_co[tmp_co['gyobae_dt'] == gyobaes_list[gyobae_num]]
            if tmp.empty:
                pass
            else:
                tmp = tmp[tmp['wk_gubun'] == 'B']
                if len(gyobaes_list) > gyobae_num+1:
                    silsan = tmp_pi[tmp_pi['GYOBAE_DT'] == gyobaes[gyobae_num+1]]['SILSAN'].values[0]
                    chongsan = tmp_pi[tmp_pi['GYOBAE_DT'] == gyobaes[gyobae_num+1]]['CHONGSAN'].values[0]
                    eudusu = tmp_pi[tmp_pi['GYOBAE_DT'] == gyobaes[gyobae_num+1]]['EUDUSU'].values[0]
                    sancha = tmp_pi[tmp_pi['GYOBAE_DT'] == gyobaes[gyobae_num+1]]['BUNMAN_SANCHA'].values[0]
                    jae = tmp_pi[tmp_pi['GYOBAE_DT'] == gyobaes[gyobae_num+1]]['JAE'].values[0]
                else:
                    sancha = None
                    if sancha == None:
                        silsan = None
                        chongsan = None
                        eudusu = None
                        jae = None
                tmp['next_bunman_sancha'] = sancha
                tmp['next_silsan'] = silsan
                tmp['next_chongsan'] = chongsan
                tmp['next_eudusu'] = eudusu
                tmp['next_jae'] = jae
                tmp['day_now_kg'] = tmp['one_now_kg']+tmp['two_now_kg']+tmp['three_now_kg']+tmp['four_now_kg']+tmp['five_now_kg']
                B_now_kg = 0
                for z in tmp['day_now_kg']:
                    B_now_kg += z
                tmp['B_now_kg'] = B_now_kg
                data = pd.concat([data,tmp], axis = 0)
#12:45
data = data.dropna()
#12:46
test = data[['farm_pig_no','gyobae_dt','one_now_kg','two_now_kg','three_now_kg','four_now_kg',
             'five_now_kg','day_now_kg','B_now_kg','next_jae','next_chongsan','next_silsan',
             'next_eudusu']]
#12:46
def get_outlier(df=None, column=None, weight=1.5):
  # target 값과 상관관계가 높은 열을 우선적으로 진행
  quantile_25 = np.percentile(df[column].values, 25)
  quantile_75 = np.percentile(df[column].values, 75)
  IQR = quantile_75 - quantile_25
  IQR_weight = IQR*weight
  lowest = quantile_25 - IQR_weight
  highest = quantile_75 + IQR_weight
  outlier_idx = df[column][ (df[column] < lowest) | (df[column] > highest) ].index
  return outlier_idx
#12:46
outlier_idx = get_outlier(df = test, column = 'next_chongsan',weight=1.5)
test.drop(outlier_idx, axis = 0, inplace = True)
#12:46
test = test[test['B_now_kg'] != 0]
#12:46
test = test.drop_duplicates(['farm_pig_no','gyobae_dt'])
import matplotlib.pyplot as plt
X = test['B_now_kg'].astype('int')
Y = test['next_chongsan'].astype('int')
plt.scatter(X,Y, alpha=0.5)
plt.xlabel('kg')
plt.ylabel('next_chongsan')
plt.show()
print('상관계수 : ',np.corrcoef(X,Y)[0,1])

TypeError: 'module' object is not callable